#### Convert a Huggingface Model (or any pytorch model) to ONNX
This guide demonstrates how to convert a Hugging Face model (or any PyTorch model) to ONNX format. We'll also show how to add a softmax layer if the model doesn't already include one.

In [ ]:
%pip install torch transformers onnx numpy

In [2]:
import torch
import torch.nn as nn
from transformers import AutoModelForImageClassification
import numpy as np

class ModelWithSoftmax(nn.Module):
    def __init__(self, original_model):
        super(ModelWithSoftmax, self).__init__()
        self.original_model = original_model
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        logits = self.original_model(x).logits
        return self.softmax(logits)  # Apply softmax to the logits

# Load the Hugging Face model
original_model = AutoModelForImageClassification.from_pretrained("google/mobilenet_v2_1.0_224")
model_with_softmax = ModelWithSoftmax(original_model)

#make sure to use the same dimensions for inference in flutter
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(
    model_with_softmax,
    dummy_input,
    "../assets/mobile.onnx",
    opset_version=11,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

C:\Users\thore\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\mobilenet_v2\modeling_mobilenet_v2.py:248: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  in_height = int(features.shape[-2])
C:\Users\thore\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\mobilenet_v2\modeling_mobilenet_v2.py:249: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  in_width = int(features.sh